In [25]:
import os

from keras.models import load_model
import cv2
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

path = "models/FullyTrained-Syn+Real/ResNetV2+Side+Height SR.keras"
side = True
height = True
test_datasets = ["BodyM"]
folder = "FullyTrained-Syn+Real"


model = load_model(path)
model_name = ".".join(((path.split("/")[-1]).split("."))[:-1])

test_folders = {"BodyM": ["testA", "testB"], 
                "Surreact-APose": ["test"]}
results = []

for dataset in tqdm(test_datasets):
    
    for folder in tqdm(test_folders[dataset]):
        size = len(os.listdir(f"Export/{dataset}/{folder}/measurements"))
        results_dataframe = pd.DataFrame(columns=["Chest", "Waist", "Pelvis", "Bicep", "Thigh", "Shoulder to wrist", "Leg", "Calf", "Wrist", "Shoulder to shoulder"]).astype([])
        
        for id in tqdm(range(size)):
            
            testing_data=[]
            y = np.load(f"../bodym-dataset/Export/measurements/Avatar_{id:06d}.npy")
            
            
            if side:
                X = np.empty(shape=(1, 320, 240, 2))
                X[0, :, :, 0] = cv2.imread(f"Export/{dataset}/{folder}/images_front/Avatar_{id:06d}.png", cv2.IMREAD_GRAYSCALE)
                X[0, :, :, 1] = cv2.imread(f"Export/{dataset}/{folder}/images_side/Avatar_{id:06d}.png", cv2.IMREAD_GRAYSCALE)
            else:
                X = np.empty(shape=(1, 320, 240, 1))
                X[0, :, :, 0] = cv2.imread(f"Export/{dataset}/{folder}/images_front/Avatar_{id:06d}.png", cv2.IMREAD_GRAYSCALE)
            testing_data.append(np.array(X))

            if height:
                if dataset == "BodyM":
                    height = y[6]
                elif dataset == "Surreact-APose":
                    height = y[16]
                testing_data.append(np.array(height).flatten())
            y_pred = model.predict(testing_data, verbose=0)
            
            if dataset == "BodyM":
                series = pd.DataFrame({
                    "Chest": abs(y_pred[0][0] - y[4]),
                    "Waist": abs(y_pred[1][0] - y[12]),
                    "Pelvis": abs(y_pred[2][0] - y[7]),
                    "Bicep": abs(y_pred[3][0] - y[2]),
                    "Thigh": abs(y_pred[4][0] - y[11]),
                    "Shoulder to wrist": abs(y_pred[5][0] - y[1]),
                    "Leg": abs(y_pred[6][0] - y[8]),
                    "Calf": abs(y_pred[7][0] - y[3]),
                    "Wrist": abs(y_pred[8][0] - y[13]),
                    "Shoulder to shoulder": abs(y_pred[9][0] - y[9])
                }, index=[0])
                
            elif dataset == "Surreact-APose":
                series = pd.DataFrame({
                            "Chest": abs(y_pred[0][0] - y[0]),
                            "Waist": abs(y_pred[1][0] - y[1]),
                            "Pelvis": abs(y_pred[2][0] - y[2]),
                            "Bicep": abs(y_pred[3][0] - y[4]),
                            "Thigh": abs(y_pred[4][0] - y[5]),
                            "Shoulder to wrist": abs(y_pred[5][0] - y[7]),
                            "Leg": abs(y_pred[6][0] - y[8]),
                            "Calf": abs(y_pred[7][0] - y[9]),
                            "Wrist": abs(y_pred[8][0] - y[11]),
                            "Shoulder to shoulder": abs(y_pred[9][0] - y[13])
                }, index=[0])
                
            
                
            results_dataframe = pd.concat([results_dataframe, series], ignore_index=True)
            
        results.append(pd.DataFrame.copy(results_dataframe))
        results_dataframe.to_csv(f"PerformanceOnReal/{model_name}-{dataset}-{folder}.csv", index=False)
        print(results_dataframe.mean())
        print(results_dataframe.mean().mean())
        
bodym_joined = pd.concat([results[0],results[1]], ignore_index=True)
bodym_joined.to_csv(f"PerformanceOnReal/{model_name}-BodyM-Whole.csv", index=False)
print(bodym_joined.mean())
print(bodym_joined.mean().mean())

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1684 [00:00<?, ?it/s]

Chest                   2.664831
Waist                   2.793806
Pelvis                  2.123273
Bicep                   1.318927
Thigh                   1.689868
Shoulder to wrist       0.944051
Leg                     1.338805
Calf                    1.183252
Wrist                   0.668813
Shoulder to shoulder    0.896125
dtype: float32
1.5621752


  0%|          | 0/1160 [00:00<?, ?it/s]

Chest                   10.796069
Waist                   12.233193
Pelvis                   9.850460
Bicep                    3.925130
Thigh                    5.986379
Shoulder to wrist        1.489775
Leg                      2.316119
Calf                     3.372875
Wrist                    1.165817
Shoulder to shoulder     1.991736
dtype: float32
5.3127556
Chest                   5.981369
Waist                   6.643906
Pelvis                  5.275013
Bicep                   2.381937
Thigh                   3.442314
Shoulder to wrist       1.166638
Leg                     1.737428
Calf                    2.076348
Wrist                   0.871529
Shoulder to shoulder    1.342999
dtype: float32
3.091948
